<a href="https://colab.research.google.com/github/VictorUceda/TrackNet-Padel/blob/main/TrackNet_Padel_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed May 25 19:28:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


import os 
path_drive = '/content/gdrive/MyDrive/Colab Notebooks/TrackNet-Padel/'
weights_path = '/content/gdrive/MyDrive/PadelModels/TrackNetWeights/TrackNet-Padel'

!pwd

Mounted at /content/gdrive
/content


In [3]:
os.chdir('/content/')
!pwd

/content


In [12]:
#!git clone https://github.com/VictorUceda/TrackNet-Padel.git
#!git clone https://github.com/Chang-Chia-Chi/TrackNet-Badminton-Tracking-tensorflow2
os.chdir('/content/TrackNet-Padel/')
!git pull

remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 19 (delta 7), reused 16 (delta 6), pack-reused 0
Unpacking objects: 100% (19/19), done.
From https://github.com/VictorUceda/TrackNet-Padel
   1aab304..69eb37d  main       -> origin/main
Updating 1aab304..69eb37d
Fast-forward
 ...PTDanish_octavos_PD_0_01_0.csv => WPTDanish_Octavos_PD_0_01_0.csv} | 0
 ...PTDanish_octavos_PD_1_20_0.csv => WPTDanish_Octavos_PD_1_20_0.csv} | 0
 ...PTDanish_octavos_PD_1_20_1.csv => WPTDanish_Octavos_PD_1_20_1.csv} | 0
 ...PTDanish_octavos_PD_1_20_2.csv => WPTDanish_Octavos_PD_1_20_2.csv} | 0
 train.py                                                              | 4 ++--
 5 files changed, 2 insertions(+), 2 deletions(-)
 rename raw_data/points/{WPTDanish_octavos_PD_0_01_0.csv => WPTDanish_Octavos_PD_0_01_0.csv} (100%)
 rename raw_data/points/{WPTDanish_octavos_PD_1_20_0.csv => WPTDanish_Octavos_PD_1_20_0.csv} (100

In [5]:
!pip3 install filterpy sktime piexif imutils Pillow focal_loss

### Test accuracy

In [ ]:
file_name = 'Final_PremierQatar_Tie'
point_n = 0
for point_n in range(9,13):
  video = 'raw_data/points/'+file_name+"_"+str(point_n)+".mp4"
  print(video)
  csv = 'raw_data/points/'+file_name+"_"+str(point_n)+".csv"
  res = 'raw_data/points/'+file_name+"_"+str(point_n)+".resPadel.txt"
  !python ../TrackNet-Padel/predict.py --video_path={video} --label_path={csv} #--load_weights={weights_path+'4'} # 2>&1 | tee {res}

raw_data/points/Final_PremierQatar_Tie_9.mp4
2022-05-25 21:43:20.779225: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Load csv file successfully
Beginning predicting......


### Train Padel

In [6]:
!mv raw_data/*.* raw_data/points/
for point_n in range(9):
  files = 'raw_data/points/*_'+str(point_n)+'\.*'
  print(files)
  !mv {files} raw_data/

raw_data/points/*_0\.*
raw_data/points/*_1\.*
raw_data/points/*_2\.*
raw_data/points/*_3\.*
raw_data/points/*_4\.*
raw_data/points/*_5\.*
raw_data/points/*_6\.*
raw_data/points/*_7\.*
raw_data/points/*_8\.*


In [8]:
!rm -rf train_data/*
!python video2img.py 

['raw_data/Final_PremierQatar_Tie_0.mp4', 'raw_data/Final_PremierQatar_Tie_1.mp4', 'raw_data/Final_PremierQatar_Tie_2.mp4', 'raw_data/Final_PremierQatar_Tie_3.mp4', 'raw_data/Final_PremierQatar_Tie_4.mp4', 'raw_data/Final_PremierQatar_Tie_5.mp4', 'raw_data/Final_PremierQatar_Tie_6.mp4', 'raw_data/Final_PremierQatar_Tie_7.mp4', 'raw_data/Final_PremierQatar_Tie_8.mp4', 'raw_data/WPTDanish_Octavos_PD_0_01_0.mp4', 'raw_data/WPTDanish_Octavos_PD_0_01_1.mp4', 'raw_data/WPTDanish_Octavos_PD_0_01_3.mp4', 'raw_data/WPTDanish_Octavos_PD_0_01_4.mp4', 'raw_data/WPTDanish_Octavos_PD_0_01_7.mp4', 'raw_data/WPTDanish_Octavos_PD_0_01_8.mp4', 'raw_data/WPTDanish_Octavos_PD_0_41_0.mp4', 'raw_data/WPTDanish_Octavos_PD_0_41_2.mp4', 'raw_data/WPTDanish_Octavos_PD_0_41_3.mp4', 'raw_data/WPTDanish_Octavos_PD_0_41_5.mp4', 'raw_data/WPTDanish_Octavos_PD_0_41_7.mp4', 'raw_data/WPTDanish_Octavos_PD_0_51_0.mp4', 'raw_data/WPTDanish_Octavos_PD_0_51_1.mp4', 'raw_data/WPTDanish_Octavos_PD_0_51_2.mp4', 'raw_data/WPTD

In [15]:
!python train.py --save_weights={weights_path+'4'} --pre_trained=True  --load_weights={weights_path+'4'} --lr=2

lerning rate: 2.0
2022-05-25 21:38:52.441643: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Beginning training......
==========Epoch 0, Train steps: 5086, Learning rate: 1.0000==========
 208/5086 [>.............................] - ETA: 14:52 - loss: 1.7699e-04 - binary_accuracy: 0.9997Traceback (most recent call last):
  File "train.py", line 53, in <module>
    verbose=1)
  File "/usr/local/lib/python3.7/dist-packages/keras/utils/traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/engine/training.py", line 1384, in fit
    tmp_logs = self.train_function(iterator)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflo